In [125]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
#from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, mean_squared_error
from modelling_functions import *

## Processing for Models

In [126]:
#### Read Data files
housing, housing_features, feat_labels, dict_dictonary = read_and_clean(filepath = "../data/train_selected_features.csv")

/Users/mdrozdov/Documents/machina_ex_kaggle/modelling/modelling_functions.py:109: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  housing.col, id_dictonary = to_numeric(housing, col, 'SalePrice')


In [127]:
htest_id, htest_features, htest_labels, htest_dictonary = read_and_clean(filepath = "../data/test_selected_features.csv", test = True, dictonary = dict_dictonary)

/Users/mdrozdov/Documents/machina_ex_kaggle/modelling/modelling_functions.py:92: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  housing.col, id_dictonary = to_numeric_test(housing, col, dictonary[col])


In [128]:
housing_features.lotarea = np.sqrt(housing_features.lotarea)
htest_features.lotarea = np.sqrt(htest_features.lotarea)
housing.saleprice = np.log(housing.saleprice)
                                 

In [129]:
housing_features.grlivarea = np.sqrt(housing_features.grlivarea)
htest_features.grlivarea = np.sqrt(htest_features.grlivarea)

In [130]:
trim = ['miscfeature', 'street', 'condition2', 'utilities', 'roofmatl', 'heating',
       'roofstyle', 'bsmtcond', 'bsmtfintype2', 'x3ssnporch','totrmsabvgrd','fireplaces','totalbsmtsf','garageyrblt']

In [131]:
#trim = ['paveddrive', 'miscfeature', 'utilities', 'roofmatl', 'heating', 'street']
training = housing_features.drop(trim, axis=1)
testing = htest_features.drop(trim, axis=1)

In [75]:
training = housing_features
testing = htest_features

In [76]:
def undo_transform(train, test, col, transform):
    if (transform == "sqrt"):
        train[col] = train[col]**2
        test[col] = test[col]**2
    elif (transform == "log"):
        train[col] = np.exp(train[col])
        test[col] = np.exp(test[col])
    #return train[col], test[col]

In [275]:
htrain, htest, ptrain, ptest = train_test_split(training, housing.saleprice, test_size = 0.2, random_state = 0,shuffle=True)

In [214]:
train_test_split?

Signature: train_test_split(*arrays, **options)
Docstring:
Split arrays or matrices into random train and test subsets

Quick utility that wraps input validation and
``next(ShuffleSplit().split(X, y))`` and application to input data
into a single call for splitting (and optionally subsampling) data in a
oneliner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float, int or None, optional (default=0.25)
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. By default, the value is set to 0.25.
    The default will change in version 0.21. It will remain 0.25 only
    if ``train_size`` is unspecified, o

## RandomForestRegressor

In [267]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 10000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 20, 30, 40]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16, 32, 64]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 1288, 2377, 3466, 4555, 5644, 6733, 7822, 8911, 10000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 29, 48, 67, 86, 105, 124, 143, 162, 181, 200, None], 'min_samples_split': [2, 5, 10, 15, 20, 30, 40], 'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64], 'bootstrap': [True, False]}


In [268]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=0, n_jobs = -1)
# Fit the random search model
rf_random.fit(htrain, ptrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.4min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [200, 1288, 2377, 3466, 4555, 5644, 6733, 7822, 8911, 10000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 29, 48, 67, 86, 105, 124, 143, 162, 181, 200, None], 'min_samples_split': [2, 5, 10, 15, 20, 30, 40], 'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [269]:
rf_random.best_params_

{'n_estimators': 3466,
 'min_samples_split': 15,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 181,
 'bootstrap': False}

In [270]:
clf = RandomForestRegressor(n_estimators=3466,
 min_samples_split=15,
 min_samples_leaf=4,
 max_features='sqrt',
 max_depth=181,
 bootstrap=False)

#sfm = SelectFromModel(clf, threshold = 0.01)
#sfm.fit(htrain, ptrain)
# ## Not Finished
# # Measure Feature Importance
#feature_selected = []
#for feature_list_index in sfm.get_support(indices=True):
#     feature_selected.append(feat_labels[feature_list_index])
#proxy = feature_selected
#trimmed = ['x1stflrsf', 'x2ndflrsf', 'garagecars', 'overallcond', 'saleprice', 'Unnamed: 0', 'bsmtfinsf1']
#testing = testing[proxy]
#housing_features = housing_features[trimmed_features]
#print(feature_selected)

In [271]:
clf.fit(htrain, ptrain)

clf.score(htest, ptest)

0.8732936491656224

In [272]:
nice = clf.predict(htest)
mean_squared_error(nice, ptest)

0.00013078731637581858

In [140]:
pred = np.exp(clf.predict(htest_features))
Submission(htest_id, pred)

## Lasso

In [216]:
alp = optimize_penalty(htrain, ptrain, model=Lasso, min_=1e-5, max_=5, step_=1000, random=False, riter=1000)

In [217]:
alp.best_params_

{'alpha': 1e-05}

In [218]:
lasso = Lasso(normalize=True)
lasso.set_params(alpha= alp.best_params_['alpha'],warm_start=True,selection='random')
lasso.fit(htrain, ptrain)

print(lasso.score(htrain, ptrain))
print(lasso.score(htest, ptest))
#print(selected_coefs)
pred = lasso.predict(htest)

0.8970408677658935
0.8678258140613786


In [233]:
mean_squared_error(pred, ptest)

0.00013412868686797256

In [234]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 0)

rf.fit(htrain, ptrain)
pred = rf.predict(htest)
mean_squared_error(pred, ptest)

0.00013224675484704436

In [174]:
import xgboost as xgb


In [230]:
grid = np.linspace(1e-5,5,1000)
error_dict = {}
for rate in grid:
    model=xgb.XGBRegressor(learning_rate=rate,verbosity=0)
    model.fit(htrain, ptrain)
    model.score(htest,ptest)
    error_dict[rate] = mean_squared_error(pred, ptest)
    

/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(dat

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [276]:
model=xgb.XGBRegressor(learning_rate=0.1670263676367637,verbosity=0)
model.fit(htrain, ptrain)
model.score(htest,ptest)
pred = model.predict(htest)
mean_squared_error(pred, ptest)

/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.00011667949762489719

In [231]:
[key for key,value in error_dict.items() if value == min(error_dict.values())]


[1e-05,
 0.005014994994994995,
 0.010019989989989991,
 0.015024984984984988,
 0.020029979979979982,
 0.025034974974974977,
 0.030039969969969976,
 0.035044964964964974,
 0.04004995995995997,
 0.045054954954954964,
 0.05005994994994996,
 0.05506494494494495,
 0.060069939939939955,
 0.06507493493493494,
 0.07007992992992994,
 0.07508492492492493,
 0.08008991991991993,
 0.08509491491491493,
 0.09009990990990992,
 0.09510490490490492,
 0.1001098998998999,
 0.10511489489489491,
 0.1101198898898899,
 0.1151248848848849,
 0.1201298798798799,
 0.1251348748748749,
 0.1301398698698699,
 0.1351448648648649,
 0.1401498598598599,
 0.14515485485485488,
 0.15015984984984987,
 0.15516484484484488,
 0.16016983983983987,
 0.16517483483483486,
 0.17017982982982988,
 0.17518482482482486,
 0.18018981981981985,
 0.18519481481481484,
 0.19019980980980986,
 0.19520480480480484,
 0.20020979979979983,
 0.20521479479479485,
 0.21021978978978983,
 0.21522478478478482,
 0.2202297797797798,
 0.22523477477477483,
 0

In [232]:
min(error_dict.values())

0.00013412868686797256

In [225]:
pred = np.exp(np.exp(model.predict(testing)))

In [226]:
pred[1:10]

array([154094.17, 183176.25, 188504.17, 182269.94, 176596.73, 167937.84,
       162444.5 , 196024.9 , 128467.05], dtype=float32)

In [227]:
Submission(htest_id, pred)